# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/tyler/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call
LOCATION = 'Pismo Beach, CA'
TERM = 'pizza'

In [14]:
## Specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER + f'{LOCATION.split(",")[0]}-{TERM}.json'
JSON_FILE

'Data/Pismo Beach-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [15]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Pismo Beach-pizza.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [ ]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [17]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term=TERM, location=LOCATION)

In [18]:
type(results)

dict

In [26]:
## How many results total?
len(results['businesses'])

20

In [20]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

- Where is the actual data we want to save?

In [27]:
results['businesses'][0]

{'id': '6NltjvK48THHxOP78m-pbA',
 'alias': 'giuseppes-cucina-italiana-pismo-beach',
 'name': "Giuseppe's Cucina Italiana",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/MVaKRVVwAEpbfLKKTIsJnQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/giuseppes-cucina-italiana-pismo-beach?adjust_creative=xQGuQUpb4cAw44Jwhkrkzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xQGuQUpb4cAw44Jwhkrkzw',
 'review_count': 1425,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.0,
 'coordinates': {'latitude': 35.14249, 'longitude': -120.64124},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '891 Price St',
  'address2': '',
  'address3': '',
  'city': 'Pismo Beach',
  'zip_code': '93449',
  'country': 'US',
  'state': 'CA',
  'display_address': ['891 Price St', 'Pismo Beach, CA 93449']},
 'phone': '+18057732870',
 'display_phone': '(805) 773-2870',
 'distance': 348.22768832

In [25]:
results['total']

89

In [28]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6NltjvK48THHxOP78m-pbA,giuseppes-cucina-italiana-pismo-beach,Giuseppe's Cucina Italiana,https://s3-media1.fl.yelpcdn.com/bphoto/MVaKRV...,False,https://www.yelp.com/biz/giuseppes-cucina-ital...,1425,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 35.14249, 'longitude': -120.64124}",[delivery],$$,"{'address1': '891 Price St', 'address2': '', '...",+18057732870,(805) 773-2870,348.227688
1,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 35.14405, 'longitude': -120.6429}","[delivery, pickup]",$$,"{'address1': '1101 Price St', 'address2': '', ...",+18057734438,(805) 773-4438,456.832734
2,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 35.1405040480287, 'longitude': -1...",[delivery],$$,"{'address1': '270 Pomeroy Ave', 'address2': ''...",+18055560347,(805) 556-0347,488.774648
3,vxFH7N_Ntxg-pagIIWM75g,beach-pizza-grover-beach,Beach Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/aCoCRY...,False,https://www.yelp.com/biz/beach-pizza-grover-be...,138,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.12111, 'longitude': -120.61964}","[delivery, pickup]",NaN,"{'address1': '960 W Grand Ave', 'address2': No...",+18054744499,(805) 474-4499,2999.014031
4,4fAGqkTIOMmkXyylyQXhBw,ginos-pizza-pismo-beach,Gino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/kPxejj...,False,https://www.yelp.com/biz/ginos-pizza-pismo-bea...,361,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 35.1351676233652, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '539 Five Cities Dr', 'address2':...",+18055560939,(805) 556-0939,1533.749594
5,kbCBKe2CcNpjy61KoZ907Q,giuseppes-express-pismo-beach,Giuseppe's Express,https://s3-media2.fl.yelpcdn.com/bphoto/goUVCf...,False,https://www.yelp.com/biz/giuseppes-express-pis...,378,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 35.14256, 'longitude': -120.64055}",[delivery],$$,"{'address1': '800 Price St', 'address2': '', '...",+18057732873,(805) 773-2873,284.396328
6,dWNxwCf8R_SDJf63E4pfAw,me-n-ed-s-pizzeria-and-craft-house-pismo-beach-2,Me n Ed’s Pizzeria and Craft House,https://s3-media2.fl.yelpcdn.com/bphoto/7fpkEX...,False,https://www.yelp.com/biz/me-n-ed-s-pizzeria-an...,65,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 35.1421, 'longitude': -120.64004}",[],NaN,"{'address1': '750 Price St', 'address2': None,...",+18055563636,(805) 556-3636,281.184189
7,NpngL0FEuj9OxaXPzqhNYg,rosas-italian-restaurant-pismo-beach-3,Rosa's Italian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/mGH8L4...,False,https://www.yelp.com/biz/rosas-italian-restaur...,1208,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 35.14017, 'longitude': -120.63844}",[delivery],$$,"{'address1': '491 Price St', 'address2': '', '...",+18057730551,(805) 773-0551,416.138931
8,EqLSwFmOJ6kosvRFELw70w,palo-mesa-wood-fired-arroyo-grande,Palo Mesa Wood Fired,https://s3-media4.fl.yelpcdn.com/bphoto/qS_y0w...,False,https://www.yelp.com/biz/palo-mesa-wood-fired-...,184,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,"{'latitude': 35.12283, 'longitude': -120.58113}",[delivery],$$,"{'address1': '150 West Branch St', 'address2':...",+18059046807,(805) 904-6807,5651.900210
9,Alnpp0cDLJ5ZsDEpzWjuWg,the-boardroom-pismo-beach-4,The Boardroom,https://s3-media1.fl.yelpcdn.com/bphoto/RXxgk7...,False,https://www.yelp.com/biz/the-boardroom-pismo-b...,279,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 35.13878, 'lo

In [37]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [44]:
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 88 previous results found.


In [39]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
n_pages

5

In [43]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                   term=TERM,
                                   offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print('[!] ERROR: ', e)


  0%|          | 0/5 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [49]:
df = pd.read_json(JSON_FILE)

In [50]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Pismo Beach-pizza.csv.gz'

In [51]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [52]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 87,158 Bytes
CSV.GZ FILE: 12,664 Bytes
the csv.gz is 6.88234365129501 times smaller!


## Next Class: Processing the Results and Mapping 